<a href="https://colab.research.google.com/github/jmartinbellido/Repsol_Python/blob/main/Repsol_Python_N3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ÍNDICE
# **ISDI Repsol D4D/D4B**
# *Notebook 3: Manipulación Avanzada*

### ÍNDICE
1. CREAR CAMPOS NUEVOS EN UN DATAFRAME
2. AGREGAR DATA
3. DATA CLEANING
4. EJERCICIOS

### Profesor: Juan Martin Bellido
* [linkedin.com/in/jmartinbellido](https://www.linkedin.com/in/jmartinbellido/)
* jmbelldo@isdi.education
---




In [68]:
# antes de empezar, cargamos librerías (instalamos en caso de ser necesario)
import pandas as pd
import numpy as np
from scipy import stats

# CREAR CAMPOS NUEVOS EN UN DATAFRAME


In [ ]:
# Importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv",index_col="Film")
df_jamesbond.dtypes # la función dtypes permite visualizar las variables incluídas en un dataframe

Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### A partir de operaciones entre campos
---

In [ ]:
# Creamos una nueva columna como resultado de una operación entre dos columnas numéricas en el dataframe
## utilizaremos la función round() para redondear el resultado
df_jamesbond["profitability"]=(df_jamesbond["Box Office"]/df_jamesbond["Budget"]).round()
df_jamesbond.head()

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary,Director/Actor,profitability
Film,,,,,,,,
Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6,Terence Young / Sean Connery,64.0
From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6,Terence Young / Sean Connery,43.0
Goldfinger,1964,Sean Connery,Guy Hamilton,820.4,18.6,3.2,Guy Hamilton / Sean Connery,44.0
Thunderball,1965,Sean Connery,Terence Young,848.1,41.9,4.7,Terence Young / Sean Connery,20.0
Casino Royale,1967,David Niven,Ken Hughes,315.0,85.0,NaN,Ken Hughes / David Niven,4.0


In [ ]:
# Creamos una nueva columna como resultado de una operación entre dos columnas de tipo texto en el dataframe
df_jamesbond["Director/Actor"]=df_jamesbond["Director"]+ " / " + df_jamesbond["Actor"]
df_jamesbond.head()

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary,Director/Actor
Film,,,,,,,
Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6,Terence Young / Sean Connery
From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6,Terence Young / Sean Connery
Goldfinger,1964,Sean Connery,Guy Hamilton,820.4,18.6,3.2,Guy Hamilton / Sean Connery
Thunderball,1965,Sean Connery,Terence Young,848.1,41.9,4.7,Terence Young / Sean Connery
Casino Royale,1967,David Niven,Ken Hughes,315.0,85.0,NaN,Ken Hughes / David Niven


### A partir de una prueba lógica
---

Creamos una nueva columna a partir de una condición *if/else* utilizando la función *np.where()*

```
np.where(condition, value if true, value if false)
```


In [ ]:
# Creamos un nuevo campo con valor condicional
## asignamos 1 o 0 en función de si el actor es Daniel Craig o no
df_jamesbond["actor_is_daniel_craig"] = np.where(df_jamesbond["Actor"]=='Daniel Craig',1,0)
df_jamesbond[["Actor","actor_is_daniel_craig"]].head()

,Actor,actor_is_daniel_craig
Film,,
Dr. No,Sean Connery,0
From Russia with Love,Sean Connery,0
Goldfinger,Sean Connery,0
Thunderball,Sean Connery,0
Casino Royale,David Niven,0


# AGREGAR DATA

In [2]:
# Importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv",index_col="Film")
df_jamesbond.dtypes

Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Introducción a agregaciones
---

En el contexto del análisis de datos, agregar data significa realizar cálculos que permitan resumir información, abstrayendo conocimiento de data que se encuentra en estado bruto. *Nota: Se trata del ejercicio que hacemos cada vez que utilizamos una tabla dinámica (pivot table) en Excel*.

Al realizar una agregación debemos establecer el tipo de cálculo que buscamos realizar. Para ello, utilizamos funciones específicas: *funciones de agregación*. 

*Funciones de agregación*

| Function    | Description                     |
|-------------|---------------------------------|
| count       | Number of non-null observations |
| nunique     | Number of unique values         |
| sum         | Sum of values                   |
| mean        | Mean of values                  |
| median      | Arithmetic median of values     |
| mode        | Mode                            |
| min         | Minimum                         |
| max         | Maximum                         |




Comenzaremos realizando agregaciones al total de las variables incluídas en un dataframe.

In [17]:
# Aplicamos una función de agregación sobre el dataframe
df_jamesbond.max()

Year                           2015
Actor                Timothy Dalton
Director              Terence Young
Box Office                    943.5
Budget                        206.3
Bond Actor Salary              17.9
dtype: object

In [ ]:
# Si quisieramos aplicar más de una función de agregación, podemos utilizar la función .agg()
df_jamesbond.agg(["max","min"])

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
max,2015,Timothy Dalton,Terence Young,943.5,206.3,17.9
min,1962,Daniel Craig,Guy Hamilton,250.9,7.0,0.6


In [ ]:
df_jamesbond.agg(["max","min"]).T # el atributo .T nos permite pivotar ("transponer") el df

,max,min
Year,2015,1962
Actor,Timothy Dalton,Daniel Craig
Director,Terence Young,Guy Hamilton
Box Office,943.5,250.9
Budget,206.3,7
Bond Actor Salary,17.9,0.6


In [42]:
# Agregamos la columna "box office", aplicando funciones min(), max() y mean()
df_jamesbond.agg({'Box Office':["min","max","mean"]})

,Box Office
min,250.900000
max,943.500000
mean,491.611538


### Agregaciones agrupadas
---

Al realizar agregaciones, frecuentemente buscamos agrupar observaciones en una o más variables categóricas. 

El método *groupby()* permite agrupar observaciones para posteriormente agregar según grupos.

```
object.groupby("column")
```


In [38]:
# Agregamos dos métricas, según una variable agrupada
df_jamesbond.groupby("Actor")[["Bond Actor Salary","Box Office"]].max()

,Bond Actor Salary,Box Office
Actor,,
Daniel Craig,14.5,943.5
David Niven,NaN,315.0
George Lazenby,0.6,291.5
Pierce Brosnan,17.9,518.5
Roger Moore,9.1,535.0
Sean Connery,5.8,848.1
Timothy Dalton,7.9,313.5


In [40]:
# Repetimos ejercicio utilizando la función .agg()
df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":"max","Box Office":"max"}
)

,Bond Actor Salary,Box Office
Actor,,
Daniel Craig,14.5,943.5
David Niven,NaN,315.0
George Lazenby,0.6,291.5
Pierce Brosnan,17.9,518.5
Roger Moore,9.1,535.0
Sean Connery,5.8,848.1
Timothy Dalton,7.9,313.5


In [41]:
# Agregamos salario máximo y media de box office, según actor
## cambiamos los nombres de las variables agregadas
df = df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":"max","Box Office":"mean"}
).sort_values("Bond Actor Salary", ascending=False)

## muchas veces, los nombres que obtenemos por defecto no son representativos de la información que almacenan
df = df.rename(columns={'Bond Actor Salary':'total_bond_salary','Box Office':'total_box_office'})

df # invocamos el objeto

,total_bond_salary,total_box_office
Actor,,
Pierce Brosnan,17.9,471.650000
Daniel Craig,14.5,691.475000
Roger Moore,9.1,422.957143
Timothy Dalton,7.9,282.200000
Sean Connery,5.8,571.114286
George Lazenby,0.6,291.500000
David Niven,NaN,315.000000


In [ ]:
# Agregamos múltiples métricas utilizando distintas funciones
## al momento de ordenar, debemos utilizar un tuple para poder especificar (i) variable y (ii) función
df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":["max","sum","mean","size"]
    ,"Budget":["max","min"]}
).sort_values(("Bond Actor Salary","max"),ascending=False) # al ordenar el df, debemos utilizar un "tuple" para definir criterio


Bond Actor Salary                       Budget       
                             max   sum       mean size    max    min
Actor                                                               
Pierce Brosnan              17.9  46.5  11.625000    4  158.3   76.9
Daniel Craig                14.5  25.9   8.633333    4  206.3  145.3
Roger Moore                  9.1  16.9   8.450000    7   91.5   27.7
Timothy Dalton               7.9  13.1   6.550000    2   68.8   56.7
Sean Connery                 5.8  20.3   3.383333    7   86.0    7.0
George Lazenby               0.6   0.6   0.600000    1   37.3   37.3
David Niven                  NaN   0.0        NaN    1   85.0   85.0

In [ ]:
# Podemos utilizar más de una variable al agregar
## en tal caso, las observaciones se agruparán según estos dos campo combinados
df_jamesbond.groupby(["Director","Actor"]).agg(
    {"Box Office":"median"}
)

Box Office
Director           Actor                     
Guy Hamilton       Roger Moore         397.15
                   Sean Connery        631.45
Irvin Kershner     Sean Connery        380.00
John Glen          Roger Moore         373.80
                   Timothy Dalton      282.20
Ken Hughes         David Niven         315.00
Lee Tamahori       Pierce Brosnan      465.40
Lewis Gilbert      Roger Moore         534.00
                   Sean Connery        514.20
Marc Forster       Daniel Craig        514.20
Martin Campbell    Daniel Craig        581.50
                   Pierce Brosnan      518.50
Michael Apted      Pierce Brosnan      439.50
Peter R. Hunt      George Lazenby      291.50
Roger Spottiswoode Pierce Brosnan      463.20
Sam Mendes         Daniel Craig        835.10
Terence Young      Sean Connery        543.80

# DATA CLEANING
En este apartado, revisaremos tres ejercicios básicos de limpieza de datos: 

*   Operar con valores nulos
*   Remover valores duplicados
*   Identificar outliers



In [43]:
# Importamos DataFrame
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv",index_col="Film")

### Operar con valores nulos
---

#### Identificar valores nulos en un DataFrame

El primer paso es corroborar si existen valores nulos en nuestros DataFrame y, en tal caso, identificar en qué campos.

En esta sección utilizaremos dos funciones/métodos:

```
isna(object)
```
> *El método isna() testea si los valores son NaN (True) o no (False)*

```
any(boolean object)
```
> *El método any() permite comprobar si al menos un elemento es True*








In [44]:
# Paso 1: aplicamos isna() al DataFrame para testear si cada elemento es nulo o no
nan_values = df_jamesbond.isna() # almacenamos el resultado en un nuevo objeto
nan_values # visualizamos el objeto creado

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
Film,,,,,,
Dr. No,False,False,False,False,False,False
From Russia with Love,False,False,False,False,False,False
Goldfinger,False,False,False,False,False,False
Thunderball,False,False,False,False,False,False
Casino Royale,False,False,False,False,False,True
You Only Live Twice,False,False,False,False,False,False
On Her Majesty's Secret Service,False,False,False,False,False,False
Diamonds Are Forever,False,False,False,False,False,False
Live and Let Die,False,False,False,False,False,True


In [45]:
# Paso 2: testeamos si al menos un elemento de cada campo es True (nulo)
nan_columns = nan_values.any() # aplicamos any() al objeto nan_values y lo almacenamos en un nuevo objeto
nan_columns # ahora visualicemos el objeto que hemos creado
# nota: al visualizar el objeto, encontramos valores nulos para la variable "Bond Actor Salary"

Year                 False
Actor                False
Director             False
Box Office           False
Budget               False
Bond Actor Salary     True
dtype: bool

#### Modificar valores nulos en un DataFrame
Una vez hemos identificado valores nulos, podemos decidir entre:
*   remover filas con al menos un valor nulo, o
*   reemplazar valores nulos

Funciones que introduciremos en esta sección:

```
fillna(value)
```
> *Este método reemplaza elementos nulos asignando el valor que establezcamos como parámetro*





In [46]:
# Hemos identificado valores nulos en campo "Bond Actor Salary", ahora procederemos a omitir filas con valores nulos en esta variable
cond = df_jamesbond["Bond Actor Salary"].isna()
df_jamesbond[-cond] # negamos la confición para omitir filas con valores nulos para variable "Bond Actor Salary"

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
Film,,,,,,
Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6
From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6
Goldfinger,1964,Sean Connery,Guy Hamilton,820.4,18.6,3.2
Thunderball,1965,Sean Connery,Terence Young,848.1,41.9,4.7
You Only Live Twice,1967,Sean Connery,Lewis Gilbert,514.2,59.9,4.4
On Her Majesty's Secret Service,1969,George Lazenby,Peter R. Hunt,291.5,37.3,0.6
Diamonds Are Forever,1971,Sean Connery,Guy Hamilton,442.5,34.7,5.8
Octopussy,1983,Roger Moore,John Glen,373.8,53.9,7.8
A View to a Kill,1985,Roger Moore,John Glen,275.2,54.5,9.1


In [47]:
# De forma alternativa podríamos optar por reeplazar valores nulos por el valor de la media para esta variable 
median_bond_salary = df_jamesbond["Bond Actor Salary"].mean() # calculamos la media
df_jamesbond["Bond Actor Salary"]=df_jamesbond["Bond Actor Salary"].fillna(median_bond_salary) # reemplazamos valores nulos

# Nota: en otros casos simplemente buscaremos reemplazar valores nulos por 0 --> .fillna(0)

### Remover valores duplicados
---

In [48]:
# Importamos un DataFrame que contiene valores duplicados
df_duplicates = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/renfe_estaciones_duplicates.csv")

#### Identificar valores duplicados

Funciones nuevas que introduciremos en esta sección:
```
object.duplicated()
```
> *Este método permite testear si una fila es duplicado perfecto de otra existente*



In [49]:
len(df_duplicates) # comprobamos número de filas en DataFrame

630

In [50]:
# Identificamos valores duplicados
cond = df_duplicates.duplicated()
df_duplicates[cond]

,CODIGO,DESCRIPCION,LATITUD,LONGITUD,DIRECCION,C.P.,POBLACION,PROVINCIA,PAIS
16,5000,GRANADA,37.184036,-3.609036,AVENIDA DE LOS ANDALUCES. S/N,18014.0,Granada,Granada,España
24,10204,ZARZALEJO,40.538817,-4.158073,CALLE DEL FERROCARRIL. S/N,28293.0,Zarzalejo,Madrid,España
57,11208,VITORIA/GASTEIZ,42.841528,-2.672665,PLAZUELA DE LA ESTACION. 1,1005.0,Vitoria-Gasteiz,Araba/Álava,España
74,11203,MANZANOS,42.742875,-2.867530,RIO ZADORRA KALEA. S/N,1220.0,Ribera Baja/Erribera Beitia,Araba/Álava,España
90,15006,CISNEROS,42.191068,-4.850206,CARRETERA P-932 - ESTACION FERROCARRIL. S/N,34320.0,Cisneros,Palencia,España
181,23004,PONTEVEDRA,42.421640,-8.635830,AVENIDA DE LA ESTACION. S/N,36003.0,Pontevedra,Pontevedra,España
222,31205,A GUDIÑA,42.060690,-7.132436,CARRETERA N-525 - BEATO SEBASTIAN APARICIO. S/N,32540.0,Gudiña. A,Ourense,España
227,34005,SAN PEDRO DEL ARROYO,40.803838,-4.871408,CALLE ESTACION FERROCARRIL. S/N,5350.0,San Pedro del Arroyo,Ávila,España
242,35206,NAVALMORAL DE LA MATA,39.894854,-5.545566,PLAZA ESTACION FERROCARRIL. 1,10300.0,Navalmoral de la Mata,Cáceres,España
253,37300,PUERTOLLANO,38.691411,-4.111611,CALLE MUELLE. S/N,13500.0,Puertollano,Ciudad Real,España


#### Remover valores duplicados

Funciones nuevas que introducidemos en esta sección:
```
object.drop_duplicates()
```
> *Este método elimina automáticamente filas duplicadas*

In [51]:
df_clean = df_duplicates.drop_duplicates() # removemos filas duplicadas y almacenamos el output en un nuevo objeto

In [52]:
len(df_clean) # corroboranos que el nuevo DataFrame contiene menos filas

620

### Identificar outliers
---


In [53]:
# Importamos DataFrame
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv",index_col="Film")

Es importante identificar outliers en variables continuas. Dependiendo del caso, estaremos interesados únicamente en detectar posibles errores o en eliminar todo posible outliers (sea un error o no).

Existen distintas formas de detectar outliers. En este curso, introduciremos un método sencillo: *normalizando la variable a z (distribución normal)*. En este método, asumiremos que la variable se distribuye normalmente y la convertiremos a z. Por convención, entendemos valores outliers a aquellos con valor absoluto mayor a 3 (z>3 y z<-3).

Funciones nuevas que introducimos en esta sección:

```
stats.zscore(column, nan_policy='omit')
```
> *Este método normaliza una variable a z (distribución normal)*





In [69]:
# Normalizamos la variable "Box Office" a z
df_jamesbond["z_Box Office"] = stats.zscore(df_jamesbond["Box Office"],nan_policy='omit').round(2)

In [70]:
# Seleccionamos "Box Office" y su versión normalizada
df_jamesbond[["Box Office","z_Box Office"]].sort_values("z_Box Office",ascending=False)
# Nota: no detectamos outliers

,Box Office,z_Box Office
Film,,
Skyfall,943.5,2.61
Thunderball,848.1,2.06
Goldfinger,820.4,1.90
Spectre,726.7,1.36
Casino Royale,581.5,0.52
From Russia with Love,543.8,0.30
Moonraker,535.0,0.25
The Spy Who Loved Me,533.0,0.24
GoldenEye,518.5,0.16


# UNIR DATAFRAMES
Con frecuencia nos vemos obligados a unir dos o más dataframes. Al unir dos dataframes, podemos estar realizando dos operaciones distintas:
*   Concatenación (unir por filas, también denominado *unions*);
*   Joins (unir columnas según algún campo en común)



In [61]:
# Importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv",index_col="Film")
df_jamesbond.dtypes

Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Unir por filas (unions)
---

Uniremos dos dataframes utilizando la función *concat()*. Nota: los datasets deben contener las mismas columnas.

In [64]:
pd.concat([df_jamesbond.head(2),df_jamesbond.tail(2)])

,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
Film,,,,,,
Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6
From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6
Skyfall,2012,Daniel Craig,Sam Mendes,943.5,170.2,14.5
Spectre,2015,Daniel Craig,Sam Mendes,726.7,206.3,NaN


### Unir por columnas (joins)
---

La función *merge()* permite unir ("join") dos dataframes según columnas. Imitando la lógica en SQL, permite elegir entre distintos criterios para el join (*left, right, inner*).

```
df_1.merge(df_2,how='inner',on=None, ...)
```

En caso de que los *keys* (columnas que usamos para unir) tengan nombres distintos en cada tabla, 

```
df_1.merge(df_2,how='inner',left_on='key',right_on='key', ...)
```


In [58]:
# importamos dataframe con data de albumes
df_albums = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_albums.csv')
df_albums.dtypes

AlbumID_Rank              int64
Artist                   object
Album                    object
Release_Year              int64
Spotify_Album            object
Description              object
wiki                     object
Duration                 object
Minutes                   int64
Seconds                   int64
Total_Seconds             int64
Label                    object
Sub_Metal_Genre          object
Rating                  float64
Rolling_Stone_Rating     object
dtype: object

In [57]:
# importamos dataframe con data de canciones
df_songs = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_songs.csv')
df_songs.dtypes

Song Index       int64
Song            object
Artist          object
Track No.        int64
AlbumID_Rank     int64
BPM              int64
Energy           int64
Dance            int64
Loud             int64
Valence          int64
Duration        object
Acoustic         int64
Popularity       int64
Spotify_Song    object
dtype: object

In [60]:
df_songs[['Song','Artist','AlbumID_Rank']].merge(
    df_albums[['AlbumID_Rank','Album','Release_Year']]  # df secundario
    ,how='left'                                         # definimos el criterio en el join, en este caso elegimos "left"
    ,left_on='AlbumID_Rank'                             # definimos la key (columna) en la tabla principal 
    ,right_on='AlbumID_Rank'                            # key en nuestra tabla secundaria
)

,Song,Artist,AlbumID_Rank,Album,Release_Year
0,War Pigs / Luke's Wall - 2014 Remaster,Black Sabbath,1,Paranoid,1970
1,Paranoid - 2016 Remaster,Black Sabbath,1,Paranoid,1970
2,Planet Caravan - 2013 Remaster,Black Sabbath,1,Paranoid,1970
3,Iron Man - 2014 Remaster,Black Sabbath,1,Paranoid,1970
4,Electric Funeral - 2013 Remaster,Black Sabbath,1,Paranoid,1970
...,...,...,...,...,...
1045,Sidewinder,Avenged Sevenfold,100,City of Evil,2005
1046,The Wicked End,Avenged Sevenfold,100,City of Evil,2005
1047,Strength of the World,Avenged Sevenfold,100,City of Evil,2005
1048,Betrayed,Avenged Sevenfold,100,City of Evil,2005


# EJERCICIOS

##### EX 1

> Dataset https://data-wizards.s3.amazonaws.com/datasets/dataset_videogames_games.csv

##### Crear un campo (*total_sales*) en el dataframe que englobe las ventas totales en todas las regiones. Crear otro campo (*video_game_segment*) que identifique videojuegos que hayan vendido en total más de 30MM ("top sales") vs. el resto ("not top sales"). Filtrar por videojuegos para la plataforma 'N64' y desplegar columnas; *name*, *platform_code*, *total_sales*, *video_game_segment*.

---



In [ ]:
import pandas as pd
df_videogames = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/dataset_videogames_games.csv")
df_videogames.dtypes

rank               int64
name              object
platform_code     object
year               int64
genre             object
publisher         object
NA_sales         float64
EU_sales         float64
JP_sales         float64
Other_sales      float64
dtype: object

In [ ]:
df_videogames["total_sales"] = df_videogames["NA_sales"] + df_videogames["EU_sales"] + df_videogames["JP_sales"] + df_videogames["Other_sales"]
df_videogames["video_game_segment"] = np.where(df_videogames["total_sales"]>30,'top_sales','not_top_sales') 

cond = df_videogames["platform_code"] == 'N64'
df_videogames[cond][['name','platform_code','total_sales','video_game_segment']].sort_values('total_sales',ascending=False)

,name,platform_code,total_sales,video_game_segment
0,Wii Sports,N64,82.74,top_sales
1,Super Mario Bros.,N64,40.24,top_sales
2,Mario Kart Wii,N64,35.83,top_sales
3,Wii Sports Resort,N64,33.00,top_sales
4,Pokemon Red/Pokemon Blue,N64,31.38,top_sales
...,...,...,...,...
15471,World Poker Tour,N64,0.01,not_top_sales
15472,Sousaku Alice to Oujisama!,N64,0.01,not_top_sales
15473,Empire Deluxe,N64,0.01,not_top_sales
15946,Spirits & Spells,N64,0.01,not_top_sales


##### EX 2 
> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv

##### EX 2.1 Calcular suma total de facturación (revenue), según sector
##### EX 2.2 Repetir el ejercicio anterior, pero filtrando únicamente por sectores Technology, Energy y Retailing
---

In [ ]:
import pandas as pd
df_fortune = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv")
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

In [ ]:
# EX 2.1
df_fortune.groupby("Sector").agg(
    {"Revenue":"sum"}
).sort_values("Revenue",ascending=False)

,Revenue
Sector,
Financials,2217159
Health Care,1614707
Energy,1517809
Retailing,1465076
Technology,1377600
"Food, Beverages & Tobacco",555967
Industrials,497581
Food and Drug Stores,483769
Motor Vehicles & Parts,482540


In [ ]:
# EX 2.2
cond = df_fortune["Sector"].isin(["Technology","Energy","Retailing"])

df_fortune[cond].groupby("Sector").agg(
    {"Revenue":"sum"}
).sort_values("Revenue",ascending=False)

,Revenue
Sector,
Energy,1517809
Retailing,1465076
Technology,1377600


##### EX 3 
> Dataset https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv

##### Extrear el top 5 planetas (homeworlds) con mayor número de personajes incluídos en el dataset.
---

In [ ]:
import pandas as pd
df_starwars_people = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv")
df_starwars_people.dtypes

name           object
height        float64
mass          float64
hair_color     object
skin_color     object
eye_color      object
birth_year    float64
gender         object
homeworld      object
species        object
sex            object
dtype: object

In [ ]:
df_starwars_people.groupby('homeworld').agg({
  "name":"nunique"   
}).rename(
    {"name":"count_characteres"}
    ,axis='columns'
).sort_values('count_characteres',ascending=False)\
.iloc[:5,]

,count_characteres
homeworld,
Naboo,11
Tatooine,10
Alderaan,3
Kamino,3
Coruscant,3


##### EX 4 
> Dataset https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv

##### Agregar las siguiente métricas según continente,

*   *Población total*
*   *PIB per cápita medio*
*   *media de % población viviendo por debajo de la línea de la pobreza*

---

In [ ]:
import pandas as pd
df_who = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv')
df_who.dtypes

Country                                                    object
CountryID                                                   int64
ContinentID                                                 int64
Adolescent fertility rate (%)                             float64
Adult literacy rate (%)                                   float64
Gross national income per capita (PPP international $)    float64
Net primary school enrolment ratio female (%)             float64
Net primary school enrolment ratio male (%)               float64
Population (in thousands) total                           float64
Population annual growth rate (%)                         float64
Population in urban areas (%)                             float64
Population living below the poverty line                  float64
Continent                                                  object
dtype: object

In [ ]:
# EX 4
output = df_who.groupby('Continent').agg({
    'Population (in thousands) total':'sum'
    ,'Gross national income per capita (PPP international $)':'mean'
    ,'Population living below the poverty line':'mean'
})

output['Population (in thousands) total'] = round(output['Population (in thousands) total'])
output['Gross national income per capita (PPP international $)'] = round(output['Gross national income per capita (PPP international $)'])
output['Population living below the poverty line'] = round(output['Population living below the poverty line'])

output.sort_values('Population (in thousands) total',ascending=False)

,Population (in thousands) total,Gross national income per capita (PPP international $),Population living below the poverty line
Continent,,,
Asia,2859153.0,2866.0,28.0
Europe,880241.0,19777.0,3.0
Africa,759147.0,3128.0,36.0
Oceania,714480.0,11716.0,12.0
South America,453480.0,7397.0,14.0
North America,441464.0,24524.0,3.0
Middle East,336867.0,14894.0,2.0


##### EX 5 
> Dataset https://data-wizards.s3.amazonaws.com/datasets/movies.csv

##### Extrear el top 10 directores con mayor media de IMDB score. Incluir únicamente directores con más de 5 películas dirigidas.
---

In [ ]:
import pandas as pd
df_movies = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/movies.csv",index_col="movie_title")
df_movies.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
movie_facebook_likes           int64
d

In [ ]:
# EX 5
output = df_movies.groupby("director_name").agg({
    "imdb_score":"mean"
    ,"movie_title":"count"
    }).rename({
    "imdb_score":"avg_imdb_score"
    ,"movie_title":"total_movies"
},axis='columns')

output[output['total_movies']>5].sort_values('avg_imdb_score',ascending=False).iloc[:10,]

,avg_imdb_score,total_movies
director_name,,
Christopher Nolan,8.425000,8
Quentin Tarantino,8.200000,8
Stanley Kubrick,8.050000,6
James Cameron,7.914286,7
Peter Jackson,7.888889,9
Alejandro G. Iñárritu,7.783333,6
David Fincher,7.750000,10
Martin Scorsese,7.660000,20
Wes Anderson,7.628571,7


##### EX 6
> Dataset https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv

##### Crear un nuevo campo ("company_size") que segmente a las empresas según cantidad de empleados según el siguiente criterio,

*   *small*, cuando la cantidad de empleados sea menor a 10000
*   *medium*, cuando la cantidad de empleados sea mayor a 10000 y menor a 100000
*   *big*, cuando la cantidad de empleados sea mayor a 100000

##### Calcular mediana de ganancias (profits), según (i) sector y (ii) tamaño de empresa ("company_size")
---

In [ ]:
import pandas as pd
import numpy as np
df_fortune = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv")
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

In [ ]:
# step 1: creating a new column based on conditions
df = df_fortune
df["company_size"] = np.where(
    df["Employees"]>100000,"big"                      # condition 1
    ,np.where(df["Employees"]>10000,"medium","small"  # condition 2
  )
)

# step 2: parsing the new variable into the groupby
df.groupby(["Sector","company_size"]).agg(
    {"Profits":"median"}
)

Profits
Sector                       company_size         
Aerospace & Defense          big            5176.0
                             medium          545.0
                             small           182.0
Apparel                      medium          415.5
                             small           174.0
Business Services            big             169.0
                             medium          419.0
                             small           148.0
Chemicals                    medium         1166.0
                             small           144.5
Energy                       medium          874.0
                             small           118.0
Engineering & Construction   medium          128.0
                             small           167.5
Financials                   big           22894.0
                             medium         1224.0
                             small           473.0
Food and Drug Stores         big            3129.5
                             medium          160.5
                             small            26.0
Food, Beverages & Tobacco    big            5452.0
                             medium          554.5
                             small           260.5
Health Care                  big            2129.0
                             medium          427.0
                             small           204.5
Hotels, Resturants & Leisure big            1293.0
                             medium          161.0
                             small         -1394.0
Household Products           big            7036.0
                             medium          325.0
                             small            89.0
Industrials                  big            2406.0
                             medium          272.5
                             small           108.0
Materials                    medium          274.0
                             small            54.0
Media                        big            8382.0
                             medium          151.0
                             small            64.0
Motor Vehicles & Parts       big            4468.0
                             medium          236.0
                             small           182.0
Retailing                    big            1165.0
                             medium          221.5
                             small            86.0
Technology                   big            7246.0
                             medium          401.0
                             small           321.0
Telecommunications           big           13345.0
                             medium          219.0
                             small           685.0
Transportation               big            4844.0
                             medium          427.0
                             small           226.0
Wholesalers                  medium          395.5
                             small            70.5

##### EX 7
> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv

##### Detectar empresas que sean outliers en términos de cantidad de empleados

In [67]:
df_fortune = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv')
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

In [71]:
df_fortune["z_Employees"] = stats.zscore(df_fortune["Employees"],nan_policy='omit').round(2)

In [76]:
cond = abs(df_fortune['z_Employees'])>3
df_fortune[cond][['Company','Employees','z_Employees']].sort_values("z_Employees",ascending=False)

,Company,Employees,z_Employees
0,Walmart,2300000,25.08
217,Yum Brands,505000,5.22
16,Kroger,431000,4.40
108,McDonald’s,420000,4.28
30,IBM,411798,4.19
27,Home Depot,385000,3.89
37,Target,341000,3.41
47,UPS,341240,3.41
10,General Electric,333000,3.32
3,Berkshire Hathaway,331000,3.30
